<a href="https://colab.research.google.com/github/DownOnCoffee/GenAI_Learning/blob/main/huggingfacetut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [1]:
!nvidia-smi

Tue Jan 20 17:37:19 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DistilBertTokenizer, DistilBertForSequenceClassification,pipeline

In [8]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
inferenceModel = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizerModel = AutoTokenizer.from_pretrained(model_name)

model = pipeline(task="sentiment-analysis",model=inferenceModel,tokenizer=tokenizerModel)
text = 'i love watching old school movies.'
print(model(text))

Device set to use cuda:0


[{'label': '5 stars', 'score': 0.753936767578125}]


Tokenization (in depth)

In [13]:
tokenizer_model = AutoTokenizer.from_pretrained('bert-base-uncased')
tokens = tokenizer_model.tokenize(text)
print(tokens)

['i', 'love', 'watching', 'old', 'school', 'movies', '.']


In [14]:
token_ids = tokenizer_model.convert_tokens_to_ids(tokens)
print(token_ids)

[1045, 2293, 3666, 2214, 2082, 5691, 1012]


In [15]:
print(tokenizer_model.decode(token_ids))

i love watching old school movies.


Fine Tuning Imdb dataset

In [16]:
!pip install datasets

In [3]:
from datasets import load_dataset
dataset = load_dataset('imdb')
dataset
dataset['train'][3]

{'text': "This film was probably inspired by Godard's Masculin, féminin and I urge you to see that film instead.<br /><br />The film has two strong elements and those are, (1) the realistic acting (2) the impressive, undeservedly good, photo. Apart from that, what strikes me most is the endless stream of silliness. Lena Nyman has to be most annoying actress in the world. She acts so stupid and with all the nudity in this film,...it's unattractive. Comparing to Godard's film, intellectuality has been replaced with stupidity. Without going too far on this subject, I would say that follows from the difference in ideals between the French and the Swedish society.<br /><br />A movie of its time, and place. 2/10.",
 'label': 0}

Padding makes short sequences longer by adding special [PAD] tokens.
With batched=True, Hugging Face passes a list of texts, not one text
Truncation shortens long sequences by cutting off extra tokens.

In [5]:
from transformers import AutoTokenizer
tmodel = AutoTokenizer.from_pretrained('bert-base-uncased')
def tokenizeContent(data):
  text = tmodel(data['text'],padding='max_length',truncation=True)
  return text
res = dataset.map(tokenizeContent,batched=True)
res




tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [8]:
from transformers import pipeline,AutoModelForSequenceClassification, TrainingArguments
train_model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    eval_strategy ="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    num_train_epochs=1,
    weight_decay=0.01,
)
training_args

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=True,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.EPOCH,
eval_use_gather_object=False,

In [9]:
from transformers import Trainer
TrainingModel=Trainer(
    model=train_model,
    args=training_args,
    train_dataset=res['train'],
    eval_dataset=res['test']
)

In [12]:
TrainingModel.train()

Epoch,Training Loss,Validation Loss


Error: You must call wandb.init() before wandb.log()